In [1]:
pwd


'/home/wsuser/work'

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_dca9a23769dd4aabb4e4935707139e1e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='8DAfFt15P683KbfAVDdWRcLDJrty6IiN7azNvRMllTJg',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_4 = client_dca9a23769dd4aabb4e4935707139e1e.get_object(Bucket='ibmcnn-donotdelete-pr-s4flyx0rst0txj', Key='Fertilizers_Recommendation_ System_For_Disease_ Prediction.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_4.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [4]:
ls

'Dataset Plant Disease'/   fruit-classification-model.tgz


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen=ImageDataGenerator(rescale=1./255,zoom_range=0.2,shear_range = 0.2,horizontal_flip=True,vertical_flip=False)
test_datagen=ImageDataGenerator(rescale=1)

In [7]:
pwd

'/home/wsuser/work'

In [8]:
x_train=train_datagen.flow_from_directory(r"/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/train",target_size=(128,128),
                                        class_mode='categorical',batch_size=32)

Found 5384 images belonging to 6 classes.


In [9]:
x_test=train_datagen.flow_from_directory(r"/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/test",target_size=(128,128),
                                        class_mode='categorical',batch_size=32)

Found 1686 images belonging to 6 classes.


In [10]:
x_train.class_indices

{'Apple___Black_rot': 0,
 'Apple___healthy': 1,
 'Corn_(maize)___Northern_Leaf_Blight': 2,
 'Corn_(maize)___healthy': 3,
 'Peach___Bacterial_spot': 4,
 'Peach___healthy': 5}

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten

In [12]:
model=Sequential()

In [13]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [14]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
model.add(Flatten())

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.add(Dense(40, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [18]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
model.fit_generator(x_train,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test),epochs=3)

/tmp/wsuser/ipykernel_241/3715613372.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test),epochs=3)


Epoch 1/3
169/169 [==============================] - 69s 407ms/step - loss: 1.6927 - accuracy: 0.3839 - val_loss: 1.4112 - val_accuracy: 0.4573
Epoch 2/3
169/169 [==============================] - 69s 407ms/step - loss: 1.4082 - accuracy: 0.4640 - val_loss: 1.2603 - val_accuracy: 0.4798
Epoch 3/3
169/169 [==============================] - 69s 407ms/step - loss: 1.2672 - accuracy: 0.4926 - val_loss: 1.2161 - val_accuracy: 0.4698


In [20]:
model.save('fruit.h5')

In [21]:
ls

'Dataset Plant Disease'/   fruit-classification-model.tgz   fruit.h5


In [22]:
!tar -zcvf fruit-classification-model.tgz fruit.h5

fruit.h5


In [35]:
!pip install watson-machine-learning-client


In [41]:
from ibm_watson_machine_learning import APIClient
wml_credetials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"_uuBmVaoXQScY286iiIVIjZ5PoWbxymkkFJiTg5VDzm2"
}

In [42]:
client = APIClient(wml_credetials)

In [43]:
client

In [45]:
def guid_space_name(client,fruit_deploy):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==fruit_deploy)['metadata']['id'])


In [46]:
space_uid =guid_space_name(client,'fruit_deploy')
print(space_uid)

1a7369cd-94df-4ab1-975a-8a142d377b78


In [47]:
client.set.default_space(space_uid)

'SUCCESS'

In [48]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [49]:
software_space_uid = client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [50]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [51]:
!tar -zcvf fruit-classification.tgz fruit.h5

fruit.h5


In [52]:
ls

'Dataset Plant Disease'/          fruit-classification.tgz
 fruit-classification-model.tgz   fruit.h5


In [53]:
model_details =  client.repository.store_model(model='fruit-classification.tgz',
                                              meta_props={
                                                  client.repository.ModelMetaNames.NAME:"CNN Model Building",
                                                  client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
                                                  client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_space_uid
                                              })

In [54]:
model_id = client.repository.get_model_id(model_details)

In [55]:
model_id

'c1da6460-c2c1-425a-b073-19abdea358cb'

In [56]:
client.repository.download(model_id,'fruit.tar.gb')

Successfully saved model content to file: 'fruit.tar.gb'


'/home/wsuser/work/fruit.tar.gb'

In [57]:
ls

'Dataset Plant Disease'/          fruit-classification.tgz   fruit.tar.gb
 fruit-classification-model.tgz   fruit.h5
